# Web content text tokenizer 

Splits titles from web articles into verbs, adjectives, adverbs, superlatives and named entities. Outputs the results to Excel.

In [1]:
import pandas as pd
import numpy as np
import re

In [5]:
!pip install spacy==2

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [471 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\spacy
  copying spacy\about.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\compat.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\glossary.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\language.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\lemmatizer.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\scorer.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\util.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\_ml.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\__init__.py -> build\lib.win-amd64-3.8\spacy
  copying spacy\__main__.py -> build\lib.win-amd64-3.8\spacy
  creating build\lib.win-amd64-3.8\spacy\cli
  copying spacy\cli\convert.py -> build\lib.win-amd64-3.8\spacy\cli
  


     ---------------------------------------- 13.2/13.2 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached murmurhash-0.28.0.tar.gz (23 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached cymem-1.31.2.tar.gz (33 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached preshed-1.0.1.tar.gz (112 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached thinc-6.10.3.tar.gz (1.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached plac-0.9.6-py2.py3-none-any.whl (20 kB)
  Using cached pathlib-1.0.1-py3-none-any.whl (14 kB)
  Using cached dill-0.2.9-py3-none-any.whl
  Using cached regex-2017.04.05.tar.gz (601 kB)
  Preparing metadata (setup.p


  copying spacy\lang\es\syntax_iterators.py -> build\lib.win-amd64-3.8\spacy\lang\es
  copying spacy\lang\es\tag_map.py -> build\lib.win-amd64-3.8\spacy\lang\es
  copying spacy\lang\es\tokenizer_exceptions.py -> build\lib.win-amd64-3.8\spacy\lang\es
  copying spacy\lang\es\__init__.py -> build\lib.win-amd64-3.8\spacy\lang\es
  creating build\lib.win-amd64-3.8\spacy\lang\fi
  copying spacy\lang\fi\stop_words.py -> build\lib.win-amd64-3.8\spacy\lang\fi
  copying spacy\lang\fi\tokenizer_exceptions.py -> build\lib.win-amd64-3.8\spacy\lang\fi
  copying spacy\lang\fi\__init__.py -> build\lib.win-amd64-3.8\spacy\lang\fi
  creating build\lib.win-amd64-3.8\spacy\lang\fr
  copying spacy\lang\fr\examples.py -> build\lib.win-amd64-3.8\spacy\lang\fr
  copying spacy\lang\fr\lemmatizer.py -> build\lib.win-amd64-3.8\spacy\lang\fr
  copying spacy\lang\fr\lex_attrs.py -> build\lib.win-amd64-3.8\spacy\lang\fr
  copying spacy\lang\fr\punctuation.py -> build\lib.win-amd64-3.8\spacy\lang\fr
  copying spacy

  
  [21 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\murmurhash
  copying murmurhash\about.py -> build\lib.win-amd64-3.8\murmurhash
  copying murmurhash\__init__.py -> build\lib.win-amd64-3.8\murmurhash
  creating build\lib.win-amd64-3.8\murmurhash\tests
  copying murmurhash\tests\test_import.py -> build\lib.win-amd64-3.8\murmurhash\tests
  copying murmurhash\tests\__init__.py -> build\lib.win-amd64-3.8\murmurhash\tests
  copying murmurhash\mrmr.pyx -> build\lib.win-amd64-3.8\murmurhash
  copying murmurhash\mrmr.pxd -> build\lib.win-amd64-3.8\murmurhash
  copying murmurhash\__init__.pxd -> build\lib.win-amd64-3.8\murmurhash
  creating build\lib.win-amd64-3.8\murmurhash\include
  creating build\lib.win-amd64-3.8\murmurhash\include\murmurhash
  copying murmurhash\include\murmurhash\MurmurHash2.h -> build\lib.win-amd64-3.8\murmurhash\include\murmurhash
  copying murmurhash\

In [2]:
input_filename  = 'data/source/find-keywords.xlsx'
output_filename = 'data/find-keywords-nouns.xlsx'

Load the articles and display a sample of the data

In [3]:
df = pd.read_excel(input_filename)
print('Article count: ', len(df))
# For testing
# df = df.head(10)
df.head(10)

Article count:  593


C:\Users\avitr\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Url,Title,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Abstract
0,NaN,ParaDocs provides our clients with the highest...,NaN,NaN,NaN,NaN,NaN
1,NaN,"AGNITY Global Inc., (AGNITY) is a global provi...",NaN,NaN,NaN,NaN,NaN
2,NaN,FlashCo has grown to be one of the largest man...,NaN,NaN,NaN,NaN,NaN
3,NaN,PAC Worldwide is a global leader in the manufa...,NaN,NaN,NaN,NaN,NaN
4,NaN,"DynexÂ® Technologies, Inc. is an original pion...",NaN,NaN,NaN,NaN,NaN
5,NaN,"Auris Surgical Robotics, Inc. is a technology ...",NaN,NaN,NaN,NaN,NaN
6,NaN,Sense Corp powers insight-driven organizations...,NaN,NaN,NaN,NaN,NaN
7,NaN,We provide quality software and uncompromising...,NaN,NaN,NaN,NaN,NaN
8,NaN,Driven by the ever-changing needs of our clien...,NaN,NaN,NaN,NaN,NaN
9,NaN,"Americo Manufacturing Company, headquartered i...",NaN,NaN,NaN,NaN,NaN


Load spacy, a Natural Language Processing tool

In [7]:
import spacy
# nlp = spacy.load('en')
spacy.__version__

'3.2.3'

In [ ]:
def tokenize_text(texts):
    docs = [doc for doc in nlp.pipe(texts, batch_size=500, n_threads=4)]
    return docs

def to_text(tokens):
    return next(map(lambda token: token.orth_, tokens), '')

def filter_first_punct(noun_chunks):
    noun_chunks = list(noun_chunks)
    if len(noun_chunks) > 0:
        print('ROOT', noun_chunks[0].sent[noun_chunks[0].start])
    return []

def get_nouns(sentences):
    return [to_text(docs.noun_chunks) for docs in sentences]

Tokenize the titles and abstracts. Token types available [here](https://spacy.io/docs/usage/pos-tagging).

In [ ]:
df['docs'] = tokenize_text(df['Title'].astype(str))

In [ ]:
def print_tokens(article_docs):
    # print('Domain:       ', url)
    print('Title:     ', article_docs)
    print('-------------')
    print('Words:     ', list(map(lambda word: word, article_docs)))
    print('Lemma:     ', list(map(lambda word: word.lemma_, article_docs)))
    print('Types:     ', list(map(lambda word: word.pos_, article_docs)))
    print('Tags:      ', list(map(lambda word: word.tag_, article_docs)))
    print('>')
    print('Nouns:     ', list(filter(lambda word: word.pos_ == 'NOUN' or word.tag_ == 'NNP' or word.tag_ == 'NNPS', article_docs)))
    print('Nouns sentences (chunks):     ', get_nouns(article_docs.sents))
    print('Noun chunks:', list(article_docs.noun_chunks))
    print('Noun chunks +1 words:', list(filter(lambda chunk: len(str(chunk).split(' ')) >= 2, list(article_docs.noun_chunks))))
    print('Verbs:     ', list(filter(lambda word: word.pos_ == 'VERB', article_docs)))
    print('Verbs Lemma:', list(map(lambda word: word.lemma_, filter(lambda word: word.pos_ == 'VERB', article_docs))))
    print('Adjectives:', list(filter(lambda word: word.pos_ == 'ADJ', article_docs)))
    print('Adjs Lemma:', list(map(lambda word: word.lemma_, filter(lambda word: word.pos_ == 'ADJ', article_docs))))
    print('Adverbs:   ', list(filter(lambda word: word.pos_ == 'ADV', article_docs)))
    print('Adverbs Lemma:', list(map(lambda word: word.lemma_, filter(lambda word: word.pos_ == 'ADV', article_docs))))
    print('Superlatives:', list(filter(lambda word: word.tag_ == 'JJS' or word.tag_ == 'RBS', article_docs)))
    print('Entities:  ', list(map(lambda entity: (entity, entity.label_), article_docs.ents)))
    
def df_url_docs(id, docs_field = 'docs'):
    return df[docs_field][id]

Manually check that the correct data types have been identified

In [ ]:
print_tokens(df_url_docs(0))

## Excel File Generation

In [ ]:
# Loops through every article and applies f() to it. 
# Then applies token_extractor() to convert from a Token to a string.
# Finally, concatenates the tokens of a single type with commas
def filter_bad_excel_strings(tokens_string):
    tokens_string = re.sub('[\000-\010]|[\013-\014]|[\016-\037]', '', tokens_string)
    if tokens_string.startswith("="):
        return tokens_string[1:]
    elif tokens_string.startswith("- "):
        return tokens_string[2:]
    else:
        return tokens_string

def map_articles(token_extractor, f, articles):
    def map_article(article):
        tokens_string = ",".join(map(token_extractor, f(article)))
        # Replace excel invalid chars
        tokens_strings = filter_bad_excel_strings(tokens_string)
        return tokens_string
    return list(map(map_article, articles))

def make_excel_df(docs_column_name = 'docs'):
    df_excel = pd.DataFrame()
    docs = df[docs_column_name]
    df_excel['Title'] = df['Title']
    df_excel['Nouns'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'NOUN' or word.tag_ == 'NNP' or word.tag_ == 'NNPS', sentence), 
                                     docs)
    df_excel['Noun Chunks (1)'] = map_articles(lambda token: token, 
                                     lambda doc : get_nouns(doc.sents), 
                                     docs)
    df_excel['Noun Chunks (2)'] = map_articles(
                                     lambda chunk: str(chunk),
                                     lambda doc : list(doc.noun_chunks), 
                                     docs)
    df_excel['Noun Chunks (3) +1 words'] = map_articles(
                                     lambda chunk: str(chunk),
                                     lambda doc : list(filter(lambda chunk: len(str(chunk).split(' ')) >= 2, list(doc.noun_chunks))), 
                                     docs)
    #df_excel['Noun Chunks +1 words (3)'] = map_articles(lambda token: token, 
    #                                 list(filter(lambda chunk: len(str(chunk).split(' ')) >= 2, list(article_docs.noun_chunks))))
    df_excel['Verbs'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'VERB', sentence), 
                                     docs)
    df_excel['Verbs Lemma'] = map_articles(lambda token: token.lemma_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'VERB', sentence), 
                                     docs)
    df_excel['Adjectives'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'ADJ', sentence), 
                                     docs)
    df_excel['Adjectives Lemma'] = map_articles(lambda token: token.lemma_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'ADJ', sentence), 
                                     docs)
    df_excel['Adverbs'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'ADV', sentence), 
                                     docs)
    df_excel['Adverbs Lemma'] = map_articles(lambda token: token.lemma_, 
                                     lambda sentence : filter(lambda word: word.pos_ == 'ADV', sentence), 
                                     docs)
    df_excel['Superlatives'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.tag_ == 'JJS' or word.tag_ == 'RBS', sentence), 
                                     docs)
    df_excel['Superlatives Lemma'] = map_articles(lambda token: token.orth_, 
                                     lambda sentence : filter(lambda word: word.tag_ == 'JJS' or word.tag_ == 'RBS', sentence), 
                                     docs)
    df_excel['Entities'] = map_articles(lambda ent: ent.orth_, 
                                     lambda sentence : sentence.ents, 
                                     docs)
    return df_excel

In [ ]:
df_excel_titles = make_excel_df()
df_excel_titles.head()

In [ ]:
writer = pd.ExcelWriter(output_filename)
df_excel_titles.to_excel(writer,'Titles')
writer.save()